<a href="https://colab.research.google.com/github/BKousha/FloorPlanSegmentation/blob/main/training_TF_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'
'Copy of Predictive Maintenance Checklist.gdoc'
 FloorPlan_samples
 model_best_val_loss_var.pkl
 Processed_Floor_Plans
 ShotBot
'SNN_soft shadow network for image compositing.pdf'
 test.txt


In [3]:
! pip install segmentation_models -q
# ! pip install -U segmentation-models==0.2.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.0 MB/s eta 0:00:00


In [4]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib as mlp


# ! pip install --upgrade keras
import segmentation_models as sm

from tensorflow.keras.saving import get_custom_objects


Segmentation Models: using `tf.keras` framework.


In [5]:
print(tf.__version__)


2.12.0


In [6]:
# Define the class mapping for merging
class_names = {
    0: "background",
    1: "room",
    2: "hallway",
    3: "kitchen",
    4: "columns",
    5: "mep",
    6: "windows",
    7: "doors",
    8: "glass_doors",
    9: "bathrooms",
    10: "elevators",
    11: "elevator_doors",
    12: "stairs",
    13: "storage",
    14: "electrical"
}
# # class_mapping  = {
# #     0:  [0],   # Background
# #     1:  [1],   # Room
# #     2:  [2],   # Hallway
# #     3:  [3],   # Kitchen
# #     4:  [4],   # Columns
# #     5:  [6],   # windows
# #     6:  [7,8], # Doors,Glass Doors
# #     7:  [9],   # Bathrooms
# #     8:  [11],  # elevator_doors
# #     9:  [5, 10, 12, 13],  # MEP, Elevators, Stairs, Storage
# #     10: [14]   # Electrical
# # }
# def translate_array(arr):
#     # Mapping dictionary for category translation
#     translation_dict = {
#         5: 9,   # Merge MEP into Bathrooms
#         10: 9,  # Merge Elevators into Bathrooms
#         12: 9,  # Merge Stairs into Bathrooms
#         13: 9,  # Merge Storage into Bathrooms
#         7: 8,   # Merge Doors into Glass Doors
#     }

#     # Create a new array by mapping the original array using the translation dictionary
#     translated_arr = np.vectorize(translation_dict.get)(arr)

#     return translated_arr

In [ ]:
# Set the necessary parameters
image_dir = "/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/temp_images"
mask_dir = "/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/temp_masks"


def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

# Load the image and mask files
image_files = []
mask_files = []
for folder_name in ['1', '2', '4', '8']:
    folder_image_dir = os.path.join(image_dir, folder_name)
    folder_mask_dir = os.path.join(mask_dir, folder_name)

    image_files.extend(sorted(listdir_fullpath(folder_image_dir)))
    mask_files.extend(sorted(listdir_fullpath(folder_mask_dir)))

print(len(image_files))
print(len(mask_files))



In [ ]:
# import cv2
# from tqdm import tqdm
# threshold=2000
# class_counts = {class_name: 0 for class_name in class_names.values()}

# for mask_file in tqdm(mask_files):
#     # Load the mask image

#     mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)

#     # Calculate the count of each class in the mask image
#     for class_value, class_name in class_names.items():
#         class_pixels = np.sum(mask == class_value)
#         if class_pixels >= threshold:
#             class_counts[class_name] += 1


# total_files = len(mask_files)
# class_percentages = {class_name: (count / total_files) * 100 for class_name, count in class_counts.items()}

# # Print the results
# for class_name, percentage in class_percentages.items():
#     print(f"Class: {class_name} - Percentage: {percentage}%")

In [ ]:
batch_size = 8
target_size = (512, 512)
num_classes = 14
Number_Category = 14

In [ ]:
class SegmentationDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_files, mask_files, batch_size, target_size, class_code,shuffle=True,background_threshold=0.9,random_state=2023):
        self.image_files = image_files
        self.mask_files = mask_files
        self.class_code = class_code
        #self.image_dir = image_dir
        #$self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        #self.num_classes = num_classes
        self.background_threshold=background_threshold
        np.random.seed(random_state)
        self.on_epoch_end()

    def __len__(self):
        return len(self.image_files) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images = []
        batch_masks = []
        batch_files = []

        for i in batch_indexes:
            image_file = self.image_files[i]
            mask_file = self.mask_files[i]


            #image_path = os.path.join(self.image_dir, image_file)
            #mask_path = os.path.join(self.mask_dir, mask_file)

            mask = tf.keras.preprocessing.image.load_img(mask_file, color_mode="grayscale", target_size=self.target_size)
            mask = tf.keras.preprocessing.image.img_to_array(mask)
            mask = self.preprocess_mask(mask)

            background_pixels = np.count_nonzero(mask == 0)
            total_pixels = mask.size
            background_percentage = background_pixels / total_pixels

            # Ignore samples where the background percentage exceeds the threshold
            if background_percentage > self.background_threshold:
                #print(background_percentage)
                continue

            image = tf.keras.preprocessing.image.load_img(image_file, color_mode="rgb", target_size=self.target_size)
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = image / 255.0

            batch_images.append(image)
            batch_masks.append(mask)
            batch_files.append(image_file)
            #print(image_file)

        return np.array(batch_images), np.array(batch_masks),batch_files
    def on_epoch_end(self):
       #return super().on_epoch_end()
       self.indexes = np.arange(len(self.image_files))
       if self.shuffle:
          np.random.shuffle(self.indexes)


    def preprocess_mask(self, mask):
        # merged_mask = np.zeros(mask.shape[:2])

        # for new_class, old_classes in class_mapping.items():
        #     merged_mask[np.isin(mask, old_classes)] = new_class
        #print(self.class_code)
        modified_mask = np.round((mask.astype(np.float64)*Number_Category/255)).astype(np.uint8)
        modified_mask = np.where(modified_mask == self.class_code, 1, 0)
        return modified_mask

In [ ]:
# Split the files into training and validation sets
train_files, val_files,mask_trin_file,mask_val_files = train_test_split(image_files, mask_files, test_size=0.1, random_state=42)
len(mask_trin_file)

In [ ]:

def plot_them(data_generator,num_batches=4):

  for _ in range(num_batches):
    sample_image, sample_mask,batch_files = data_generator[_]

    for i in range(batch_size):
      # Plot the original image and the predicted mask
      try:
        img=sample_image[i]
        #image_array_scaled = np.round((sample_mask[i].astype(np.float64)*Number_Category/255)).astype(np.uint8)
        msk=sample_mask[i]
        txt=batch_files[i]
        print(txt)

        fig, axes = plt.subplots(1, 3, figsize=(10, 5))
        axes[0].imshow(img)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        axes[1].imshow(msk)
        axes[1].set_title('Mask')
        axes[1].axis('off')

        axes[2].imshow(img)
        axes[2].axis('off')
        axes[2].imshow(msk,alpha=0.6)#, cmap=cmap,norm=norm
        axes[2].set_title('Original Image and mask overlaped')

        #axes[2].legend(handles=legend_patches, loc='upper left', bbox_to_anchor=(1.05, 1))


        plt.tight_layout()
        plt.show()
        j=j+1
        print(j)
      except:
        pass



In [ ]:
train_data_generators=[]
val_data_generators=[]
for i in range(1,14):
    #tmp_train_data_generator = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=i,background_threshold=0.99)
    tmp_train_data_generator = SegmentationDataGenerator(train_files, mask_trin_file, batch_size, target_size, class_code=i,shuffle=True,background_threshold=0.99)
    tmp_val_data_generator   = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=i,shuffle=False,background_threshold=0.99)
    #plot_them(val_data_generator_1,num_batches=1)
    train_data_generators.append(tmp_train_data_generator)
    val_data_generators.append(tmp_val_data_generator)


In [ ]:
print(batch_size)
for _ in range(4):
    sample_image, sample_mask,batch_files = train_data_generators[0][_]
    print(sample_image.shape,sample_mask.shape)

# Rooms

In [ ]:
#val_data_generator_1 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=1,background_threshold=0.99)
#plot_them(val_data_generator_1,num_batches=1)

# Hallways

In [ ]:
# val_data_generator_2 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=2,background_threshold=0.99)
# plot_them(val_data_generator_2,num_batches=1)

# Kitchens

In [ ]:
# val_data_generator_3 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=3,background_threshold=0.99)
# plot_them(val_data_generator_3)

# Columns

In [ ]:
# val_data_generator_4 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=4,background_threshold=0.99)
# plot_them(val_data_generator_4)

# MEPs

In [ ]:
# val_data_generator_5 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=5,background_threshold=0.99)
# plot_them(val_data_generator_5,num_batches=20)

# Windows

In [ ]:
# val_data_generator_6 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=6,background_threshold=0.99)
# plot_them(val_data_generator_6)

# doors

In [ ]:
# val_data_generator_7 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=7,background_threshold=0.99)
# plot_them(val_data_generator_7)


# glass_doors

In [ ]:
# val_data_generator_8 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=8,background_threshold=0.99)
# plot_them(val_data_generator_8,num_batches=20)



# bathrooms

In [ ]:
# val_data_generator_9 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=9,background_threshold=0.99)
# plot_them(val_data_generator_9,num_batches=10)



# Elevators

In [ ]:
# val_data_generator_10 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=10,background_threshold=0.99)
# plot_them(val_data_generator_10,num_batches=10)



# Elevator doors

In [ ]:
# val_data_generator_11 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=11,background_threshold=0.99)
# plot_them(val_data_generator_11,num_batches=20)

# Stairs

In [ ]:
# val_data_generator_12 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=12,background_threshold=0.99)
# plot_them(val_data_generator_12,num_batches=5)

# Storage

In [ ]:
# val_data_generator_13 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=13,background_threshold=0.90)
# plot_them(val_data_generator_13,num_batches=5)

# electrical

In [ ]:
# val_data_generator_14 = SegmentationDataGenerator(val_files, mask_val_files, batch_size, target_size, class_code=14,background_threshold=0.90)
# plot_them(val_data_generator_14,num_batches=50)

In [ ]:
# https://tensorlayer.readthedocs.io/en/latest/_modules/tensorlayer/cost.html#dice_coe
def dice_coe(output, target, axis = None, smooth=1e-10):
    output = tf.dtypes.cast( tf.math.greater(output, 0.5), tf. float32 )
    target = tf.dtypes.cast( tf.math.greater(target, 0.5), tf. float32 )
    inse = tf.reduce_sum(output * target, axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)

    dice = (2. * inse + smooth) / (l + r + smooth)
    dice = tf.reduce_mean(dice, name='dice_coe')
    return dice

# https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu
def tversky(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)
def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)

get_custom_objects().update({"focal_tversky": focal_tversky_loss})

In [ ]:
tf.keras.backend.set_image_data_format('channels_last')

model = sm.Unet('resnet34', classes=1, activation='sigmoid')
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.01),
                      loss = tf.keras.losses.BinaryCrossentropy(),#'focal_tversky',
                      metrics=[dice_coe,'accuracy'])


In [ ]:
# Train the model using the data generators
history=model.fit(train_data_generators[0], epochs=10, validation_data=val_data_generators[0])


In [ ]:
# Load the best saved model
best_model = tf.keras.models.load_model("best_model.h5")

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_data_generators[0])
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)


In [ ]:
# Get the loss values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss values
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss Function')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()